In [2]:
import pandas as pd

#原始数据
df = pd.read_csv("clean_kanzhun_corp_info0.csv")
del df['Unnamed: 0']
len(df['name']) #90
df.head(10) #显示部分数据

,name,salary,score,type,location,scale,url
0,万达影城,平均工资 ￥3059,4.0,影视/媒体,大连,10000人以上,www.wandafilm.com
1,蓝色光标,平均工资 ￥6362,3.8,广告/公关/会展,北京,1000-9999人,www.bluefocusgroup.com
2,奥美广告,平均工资 ￥5393,4.1,广告/公关/会展,上海,1000-9999人,www.ogilvy.com.cn/
3,电通,平均工资 ￥7000,3.7,广告/公关/会展,北京,500-999人,www.dentsu.com.cn
4,中央电视台,平均工资 ￥7975,4.1,影视/媒体,北京,1000-9999人,www.cctv.com
5,ABC公司,平均工资 ￥7426,4.1,影视/媒体,北京,100-499人,abcnews.go.com/
6,华数传媒,平均工资 ￥4620,3.6,影视/媒体,杭州,1000-9999人,www.wasu.com.cn
7,分众传媒,平均工资 ￥5000,3.6,影视/媒体,北京,1000-9999人,www.focusmedia.cn
8,上海奥维思市场营销服务公司,平均工资 ￥3625,2.6,广告/公关/会展,上海,20-99人,www.alwaysmkt.com
9,天津天士力医药营销集团有限公司,平均工资 ￥5500,3.5,广告/公关/会展,天津,20-99人,www.tasly.com/fgs/yiyao/index.htm


### 传媒公司概况

#### (1)薪资水平

In [3]:
len(df['salary']) #90
df['new_salary'] = df['salary'].str.lstrip("平均工资  ￥")

for i in range(0,90):
    df['new_salary'][i] = int(df['new_salary'][i])
df2 = df.sort_values(by='new_salary', ascending=False)[:20]

In [4]:
import pandas as pd
import numpy as np
df['new_salary']=pd.DataFrame(df['new_salary'],dtype=np.int)
df['new_salary'].describe()

count       90.000000
mean      5413.711111
std       2104.214648
min       1300.000000
25%       4237.500000
50%       5515.000000
75%       6320.000000
max      15840.000000
Name: new_salary, dtype: float64

In [5]:
import plotly.plotly as py
import plotly.graph_objs as go

random_x = df2['name'][:20]
random_y0 = df2['new_salary'][:20]
random_y1 = df2['score'][:20]


# Create a trace
trace0 = go.Scatter(
    x = random_x,
    y = random_y0,
    mode = 'lines+markers',
    name = 'SALARY'
)
trace1 = go.Scatter(
    x = random_x,
    y = random_y1,
    mode = 'lines+markers',
    name = 'GRADE'
)

data = [trace0, trace1]
py.iplot(data, filename='line-mode')

#### （2）地域分布

In [6]:
df['location'].value_counts()

北京     38
上海     18
广州     11
深圳      5
杭州      2
成都      2
武汉      2
重庆      2
长沙      2
天津      2
无锡      1
济南      1
南京      1
湖州      1
苏州      1
大连      1
Name: location, dtype: int64

In [35]:
import plotly.plotly as py
import plotly.graph_objs as go

pd_city = pd.read_csv("region_comm_corp_info.csv")
data = [go.Bar(
            x= pd_city['location'],
            y= pd_city['count']
    )]

py.iplot(data, filename='basic-bar')

In [52]:
import plotly.plotly as py
import plotly.graph_objs as go

import pandas as pd

mapbox_access_token = 'pk.eyJ1IjoiaWlpamVubnkiLCJhIjoiY2pwOGJkaDZjMTdqMzNwcDRndWV4Y2IzaCJ9.xprdAkm3-TNaO1MzkvjLyw'

df_location = pd.read_csv("region_comm_corp_info.csv")
site_lat = df_location.lat
site_lon = df_location.lon
locations_name = df_location.location

data = [
    go.Scattermapbox(
        lat=site_lat,
        lon=site_lon,
        mode='markers',
        marker=dict(
            size=17,
            color='rgb(255, 0, 0)',
            opacity=0.7
        ),
        text=locations_name,
        hoverinfo='text'
    ),
    go.Scattermapbox(
        lat=site_lat,
        lon=site_lon,
        mode='markers',
        marker=dict(
            size=8,
            color='rgb(242, 177, 172)',
            opacity=0.7
        ),
        hoverinfo='none'
    )]

layout = go.Layout(
    title='Geographical distribution of media companies',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=38,
            lon=-94
        ),
        pitch=0,
        zoom=3,
        style='light'
    ),
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='传媒公司分布图')

### 传媒公司类型分布

In [9]:
# 拆分type信息，并放在同一列上
types = df['type'].str.split('/', expand=True).stack().reset_index(level=1, drop=True)#.rename('type')
types.index = range(1,len(types) + 1) #重新index
len(types) #229
types.value_counts()

媒体     41
影视     41
公关     34
广告     34
会展     34
文化     15
艺术     15
出版     15
dtype: int64

In [10]:
df['type'].value_counts()

影视/媒体        41
广告/公关/会展     34
文化/艺术/出版     15
Name: type, dtype: int64

In [11]:
import plotly.plotly as py
import plotly.graph_objs as go

fig = {
  "data": [
    {
      "values": [41,34,15],
      "labels": [
        "影视/媒体",
        "广告/公关/会展",
        "文化/艺术/出版",
      ],
      "domain": {"x": [0, .48]},
      "name": "The proportion of types of media company",
      "hoverinfo":"label+percent+name",
      "hole": .4,
      "type": "pie"
    },
  ],
  "layout": {
        "title":"The proportion of types of media company",
        "annotations": [
            {
                "font": {
                    "size": 15
                },
                "showarrow": False,
                "text": "type",
                "x": 0.20,
                "y": 0.5
            },
        ]
    }
}
py.iplot(fig, filename='hot_corp_proportion')

###  媒体/影视,公关/会展/广告,文化／艺术／出版 三者之间的比较

#### (1) 薪资比较

In [12]:
df_media = df[df['type'] == '影视/媒体 ']
df_ad = df[df['type'] == '广告/公关/会展 ']
df_pub = df[df['type'] == '文化/艺术/出版 ']

In [29]:
import pandas as pd
import numpy as np
df['new_salary']=pd.DataFrame(df['new_salary'],dtype=np.int)

df_media['new_salary'].describe()

count       41.000000
mean      5671.073171
std       2376.656626
min       1300.000000
25%       4495.000000
50%       5800.000000
75%       6320.000000
max      15840.000000
Name: new_salary, dtype: float64

In [30]:
df_ad['new_salary'].describe()

count      34.000000
mean     5105.558824
std      1926.561047
min      1738.000000
25%      3531.250000
50%      5393.000000
75%      6346.500000
max      9254.000000
Name: new_salary, dtype: float64

In [31]:
df_pub['new_salary'].describe()

count      15.000000
mean     5408.733333
std      1683.926723
min      2000.000000
25%      5275.500000
50%      5530.000000
75%      6045.000000
max      8000.000000
Name: new_salary, dtype: float64

#### (2)评分比较

In [16]:
df_media
df_media.index = range(1,len(df_media) + 1)
df_media.to_csv("df_media.csv")

df_ad
df_ad.index = range(1,len(df_ad) + 1)
df_ad.to_csv("df_ad.csv")

df_pub
df_pub.index = range(1,len(df_pub) + 1)
df_pub.to_csv("df_pub.csv")

In [17]:
df_media0 = pd.read_csv("df_media.csv")
df_media0['score'].describe()

count    41.000000
mean      3.719512
std       0.398886
min       3.000000
25%       3.500000
50%       3.700000
75%       4.000000
max       5.000000
Name: score, dtype: float64

In [18]:
df_pub0 = pd.read_csv("df_pub.csv")
df_pub0['score'].describe()

count    15.000000
mean      3.673333
std       0.458984
min       2.300000
25%       3.600000
50%       3.700000
75%       4.000000
max       4.100000
Name: score, dtype: float64

In [19]:
df_ad0 = pd.read_csv("df_ad.csv")
df_ad0.drop(df_ad0.index[13],axis=0,inplace=True)
df_ad0.to_csv("df_ad0.csv")
df_ad0 = pd.read_csv("df_ad0.csv")
df_ad0['score'].describe()

count    33.000000
mean      3.630303
std       0.490844
min       2.500000
25%       3.400000
50%       3.700000
75%       4.000000
max       5.000000
Name: score, dtype: float64

In [24]:
import pandas as pd
#原始数据
df00 = pd.read_csv("salary_grade.csv")
df00

,name,mean,min,max,75%,grade,max_g
0,media,5671,1300,15840,6320,3.719,5.0
1,ad,5106,1738,9254,6346,3.673,4.1
2,pub,5409,1683,8000,6045,3.630,5.0


In [34]:
import plotly.plotly as py
import plotly.graph_objs as go


trace1 = go.Bar(
    x=['Meida', 'Ad', 'Pub'],
    y=[5671, 5106, 5409],
    name='salary_mean'
)
trace3 = go.Bar(
    x=['Meida', 'Ad', 'Pub'],
    y=[1300, 1738, 1683],
    name='salary_min'
)
trace4 = go.Bar(
    x=['Meida', 'Ad', 'Pub'],
    y=[15840, 9254, 8000],
    name='salary_max'
)
trace2 = go.Bar(
    x=['Meida', 'Ad', 'Pub'],
    y=[3.719, 3.673, 3.630],
    name='grade_mean'
)

data = [trace1, trace3, trace4, trace2]
layout = go.Layout(
    barmode='group'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='grouped-bar')

#### (3)地域比较

In [21]:
df_media['location'].value_counts()

北京     16
上海      7
广州      3
杭州      2
武汉      2
长沙      2
深圳      2
无锡      1
成都      1
南京      1
济南      1
重庆      1
天津      1
大连      1
Name: location, dtype: int64

In [22]:
df_ad['location'].value_counts()

北京     14
上海     10
广州      5
深圳      2
湖州      1
苏州      1
天津      1
Name: location, dtype: int64

In [23]:
df_pub['location'].value_counts()

北京     8
广州     3
深圳     1
成都     1
重庆     1
上海     1
Name: location, dtype: int64

In [55]:
import plotly.plotly as py
import plotly.graph_objs as go

import pandas as pd

mapbox_access_token = 'pk.eyJ1IjoiaWlpamVubnkiLCJhIjoiY2pwYW53N3IxMXhmdTNwbzN0MGN4YTM3aCJ9.YIJO9O_U89eF4k_NeEKwGw'

df_location = pd.read_csv("ad.csv")
site_lat1 = df_location.lat1
site_lon1 = df_location.lon1
locations_name1 = df_location.ad

data1 = [
    go.Scattermapbox(
        lat=site_lat1,
        lon=site_lon1,
        mode='markers',
        marker=dict(
            size=17,
            color='rgb(255, 0, 0)',
            opacity=0.7
        ),
        text=locations_name1,
        hoverinfo='text'
    ),
    go.Scattermapbox(
        lat=site_lat1,
        lon=site_lon1,
        mode='markers',
        marker=dict(
            size=8,
            color='rgb(242, 177, 172)',
            opacity=0.7
        ),
        hoverinfo='none'
    )]



layout = go.Layout(
    title='Geographical distribution of Ad companies',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=38,
            lon=-94
        ),
        pitch=0,
        zoom=3,
        style='light'
    ),
)


fig1 = dict(data=data1, layout=layout)
py.iplot(fig1, filename='传媒公司分布图')

In [58]:
import plotly.plotly as py
import plotly.graph_objs as go

import pandas as pd

mapbox_access_token = 'pk.eyJ1IjoiaWlpamVubnkiLCJhIjoiY2pwYW9hemk3MDJ1ZzN5cGhjdnMzemRpZyJ9.NBLGmbKVxTbobt4ekO9pOA'

df_location = pd.read_csv("pub.csv")
site_lat2 = df_location.lat2
site_lon2 = df_location.lon2
locations_name2 = df_location.pub

data2 = [
    go.Scattermapbox(
        lat=site_lat2,
        lon=site_lon2,
        mode='markers',
        marker=dict(
            size=17,
            color='rgb(0, 51, 204)',
            opacity=0.7
        ),
        text=locations_name2,
        hoverinfo='text'
    ),
    go.Scattermapbox(
        lat=site_lat2,
        lon=site_lon2,
        mode='markers',
        marker=dict(
            size=8,
            color='rgb(0, 153, 255)',
            opacity=0.7
        ),
        hoverinfo='none'
    )]

layout = go.Layout(
    title='Geographical distribution of Publication companies',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=38,
            lon=-94
        ),
        pitch=0,
        zoom=3,
        style='light'
    ),
)

#fig1 = dict(data=data1, layout=layout)
fig2 = dict(data=data2, layout=layout)
py.iplot(fig2, filename='传媒公司分布图')

In [57]:
import plotly.plotly as py
import plotly.graph_objs as go

import pandas as pd

mapbox_access_token = 'pk.eyJ1IjoiaWlpamVubnkiLCJhIjoiY2pwOGJkaDZjMTdqMzNwcDRndWV4Y2IzaCJ9.xprdAkm3-TNaO1MzkvjLyw'

df_location = pd.read_csv("media.csv")
site_lat3 = df_location.lat3
site_lon3 = df_location.lon3
locations_name3 = df_location.media

data3 = [
    go.Scattermapbox(
        lat=site_lat3,
        lon=site_lon3,
        mode='markers',
        marker=dict(
            size=17,
            color='rgb(255, 153, 102)',
            opacity=0.7
        ),
        text=locations_name3,
        hoverinfo='text'
    ),
    go.Scattermapbox(
        lat=site_lat3,
        lon=site_lon3,
        mode='markers',
        marker=dict(
            size=8,
            color='rgb(255, 255, 153)',
            opacity=0.7
        ),
        hoverinfo='none'
    )]

layout = go.Layout(
    title='Geographical distribution of media companies',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=38,
            lon=-94
        ),
        pitch=0,
        zoom=3,
        style='light'
    ),
)


fig3 = dict(data=data3, layout=layout)
py.iplot(fig3, filename='传媒公司分布图')